In [61]:
import pandas as pd
import lifelines
import numpy as np
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
from sklearn.preprocessing import OneHotEncoder


In [62]:
df = pd.read_csv('FHS_Network_features_dementia_survival_data_wave7.csv',index_col=0).reset_index(drop=True)

In [63]:
df.columns

Index(['idr', 'SEX', 'AGE7', 'BMI7', 'CURRSMK7', 'SBP7', 'DBP7', 'CALC_LDL7',
       'HDL7', 'TC7', 'TRIG7', 'DATE7', 'idtype', 'DEM_STATUS', 'DEM_SURVDATE',
       'AD_STATUS', 'VAD_STATUS', 'DENSITY', 'CONSTRAINT', 'EFFECTIVE_SIZE',
       'DEGREE', 'DEGREE_CENTRALITY', 'EIGENVECTOR', 'KATZ',
       'CLOSENESS_CENTRALITY', 'BETWEENNESS_CENTRALITY', 'CLUSTCOEF', 'APOE',
       'APOE_risk', 'CURR_DIAB7', 'Death_status_at_followup',
       'somecoll_education', 'HS_education', 'education_4level', 'CES-D_score',
       'CSD-D_loneliness_item', 'AGE35', 'AGE65', 'AGE_Q', 'AGE_10',
       'sharealterid', 'RELTYPE', 'FAMILIAL', 'ALTERTYPE', 'alteridtype',
       'SPELLBEGIN', 'SPELLEND', 'EDUCG', 'CESD_TOT', 'CESD_16', 'CESD_20',
       'CESD_LONELINESS', 'LONELY_3', 'SNI_TOT', 'SNI_2', 'DIST1_5', 'DIST7_5',
       'SEX_SHANNON', 'RELTYPE_SHANNON', 'ALTERTYPE_SHANNON', 'AGE35_SHANNON',
       'AGE65_SHANNON', 'AGE_Q_SHANNON', 'AGE_10_SHANNON', 'SNI_TOT_SHANNON',
       'SNI_2_SHANNON', 'EDU

In [78]:
network_features = ['DEGREE',
                    'DEGREE_CENTRALITY', 
                    'CONSTRAINT',
                    'EFFECTIVE_SIZE',
                    'EIGENVECTOR',
                    'KATZ',
                    'CLOSENESS_CENTRALITY',
                    'BETWEENNESS_CENTRALITY',
                    'CLUSTCOEF',
                    'AGE7',
                    'SEX',
                    'somecoll_education',
                    'HS_education',
                    'APOE_risk',
                    'FAMILIAL_RELATIVE_RATIO','APOE4_+_RATIO','DIST1_5_NEAR_RATIO','DIST7_5_NEAR_RATIO'
                   ]

modifiers = [c for c in df.columns if 'SHANNON'  in c]



In [79]:
df.columns

Index(['idr', 'SEX', 'AGE7', 'BMI7', 'CURRSMK7', 'SBP7', 'DBP7', 'CALC_LDL7',
       'HDL7', 'TC7', 'TRIG7', 'DATE7', 'idtype', 'DEM_STATUS', 'DEM_SURVDATE',
       'AD_STATUS', 'VAD_STATUS', 'DENSITY', 'CONSTRAINT', 'EFFECTIVE_SIZE',
       'DEGREE', 'DEGREE_CENTRALITY', 'EIGENVECTOR', 'KATZ',
       'CLOSENESS_CENTRALITY', 'BETWEENNESS_CENTRALITY', 'CLUSTCOEF', 'APOE',
       'APOE_risk', 'CURR_DIAB7', 'Death_status_at_followup',
       'somecoll_education', 'HS_education', 'education_4level', 'CES-D_score',
       'CSD-D_loneliness_item', 'AGE35', 'AGE65', 'AGE_Q', 'AGE_10',
       'sharealterid', 'RELTYPE', 'FAMILIAL', 'ALTERTYPE', 'alteridtype',
       'SPELLBEGIN', 'SPELLEND', 'EDUCG', 'CESD_TOT', 'CESD_16', 'CESD_20',
       'CESD_LONELINESS', 'LONELY_3', 'SNI_TOT', 'SNI_2', 'DIST1_5', 'DIST7_5',
       'SEX_SHANNON', 'RELTYPE_SHANNON', 'ALTERTYPE_SHANNON', 'AGE35_SHANNON',
       'AGE65_SHANNON', 'AGE_Q_SHANNON', 'AGE_10_SHANNON', 'SNI_TOT_SHANNON',
       'SNI_2_SHANNON', 'EDU

In [80]:
##for diversity indices, change anything >0 as 1
for c in modifiers:
    df[c] = df[c].apply(lambda x: 1 if x>0 else 0)
    
    

In [81]:
modifiers

['SEX_SHANNON',
 'RELTYPE_SHANNON',
 'ALTERTYPE_SHANNON',
 'AGE35_SHANNON',
 'AGE65_SHANNON',
 'AGE_Q_SHANNON',
 'AGE_10_SHANNON',
 'SNI_TOT_SHANNON',
 'SNI_2_SHANNON',
 'EDU_COLLEGE_SHANNON',
 'EDU_HS_SHANNON']

In [82]:
[modifiers.append(x) for x in ['SEX','AGE7','APOE_risk','somecoll_education','HS_education','CVD_HISTORY','CES-D_score','CSD-D_loneliness_item','FAMILIAL_RELATIVE_RATIO','APOE4_+_RATIO','DIST1_5_NEAR_RATIO','DIST7_5_NEAR_RATIO']]


[None, None, None, None, None, None, None, None, None, None, None, None]

In [83]:
# Define Event and time.to.event variables
df['event'] = df['DEM_STATUS']
df['time_to_event'] = df['DEM_SURVDATE']

df_cox = df[['event','time_to_event']]


In [84]:
#keep event as dem status
# df_COX <- within(df_COX, Event[Event == 0 & Death_status_at_followup == 1] <- 0)
# # Same for anyone who did  NOT die or did NOT have dementia.  
# df_COX <- within(df_COX, Event[Event == 0 & Death_status_at_followup == 0] <- 0)
# # Anyone with dementia (regardless of death status) is coded as an event (1).
# df_COX <- within(df_COX, Event[Event == 1 & Death_status_at_followup == 1] <- 1)
# df_COX <- within(df_COX, Event[Event == 1 & Death_status_at_followup == 0] <- 1)

In [85]:

cph = CoxPHFitter()

results = []
for f in network_features:
    for m in modifiers:
        if m!= f:
            df_cox['interaction_term'] = df[f]*df[m]
            try:
                cph.fit(df_cox, duration_col='time_to_event', event_col='event')
                res= cph.summary.reset_index().replace({'interaction_term':f+'*'+m})
                res['term 1'] = f
                res['term 2'] = m
                results.append(res)
                if  cph.summary['p'].values[0]<.05:
                    print('-'*50)
                    print(f+' * '+m)
                    print('-'*50)
                    cph.print_summary()
            except Exception as e:
                print(e)
        
# results_df = pd.DataFrame({'feature':[],'modifier':[],'coef':[],'pval':[]})

/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model


/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model


/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:794: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65y

...

<lifelines.CoxPHFitter: fitted with 997 total observations, 846 right-censored observations>
             duration col = 'time_to_event'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 997
number of events observed = 151
   partial log-likelihood = -964.71
         time fit was run = 2021-11-16 21:22:11 UTC

---
                   coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                
interaction_term   0.03       1.03       0.02             0.00             0.06                 1.00                 1.07

                    z    p   -log2(p)
covariate                            
interaction_term 2.02 0.04       4.52
---
Concordance = 0.55
Partial AIC = 1931.42
log-likelihood ratio test = 4.05 on 1 df
-log2(p) of ll-ratio test = 4.50

/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]


--------------------------------------------------
DIST7_5_NEAR_RATIO * CSD-D_loneliness_item
--------------------------------------------------


<lifelines.CoxPHFitter: fitted with 997 total observations, 846 right-censored observations>
             duration col = 'time_to_event'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 997
number of events observed = 151
   partial log-likelihood = -964.84
         time fit was run = 2021-11-16 21:22:12 UTC

---
                   coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                
interaction_term   0.34       1.41       0.16             0.03             0.66                 1.03                 1.94

                    z    p   -log2(p)
covariate                            
interaction_term 2.12 0.03       4.88
---
Concordance = 0.54
Partial AIC = 1931.68
log-likelihood ratio test = 3.79 on 1 df
-log2(p) of ll-ratio test = 4.28

/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_1723/3820400341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [86]:
results_df = pd.concat(results)
results_df.to_csv('modification_effects.csv')

In [87]:
results_df[results_df['term 2']=='DIST7_5_NEAR_RATIO']

,covariate,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p),term 1,term 2
0,DEGREE*DIST7_5_NEAR_RATIO,0.031953,1.032469e+00,0.021837,-0.010847,0.074753,9.892114e-01,1.077618e+00,1.463233,0.143404,2.801846,DEGREE,DIST7_5_NEAR_RATIO
0,DEGREE_CENTRALITY*DIST7_5_NEAR_RATIO,146.887089,6.198007e+63,100.385314,-49.864511,343.638688,2.208606e-22,1.739346e+149,1.463233,0.143404,2.801846,DEGREE_CENTRALITY,DIST7_5_NEAR_RATIO
0,CONSTRAINT*DIST7_5_NEAR_RATIO,-0.001266,9.987345e-01,0.258956,-0.508810,0.506278,6.012104e-01,1.659104e+00,-0.004890,0.996098,0.005640,CONSTRAINT,DIST7_5_NEAR_RATIO
0,EFFECTIVE_SIZE*DIST7_5_NEAR_RATIO,0.043289,1.044240e+00,0.038579,-0.032324,0.118902,9.681930e-01,1.126259e+00,1.122097,0.261821,1.933345,EFFECTIVE_SIZE,DIST7_5_NEAR_RATIO
0,EIGENVECTOR*DIST7_5_NEAR_RATIO,2.120021,8.331313e+00,4.381061,-6.466700,10.706742,1.554346e-03,4.465593e+04,0.483906,0.628453,0.670124,EIGENVECTOR,DIST7_5_NEAR_RATIO
0,KATZ*DIST7_5_NEAR_RATIO,4.399511,8.141104e+01,3.137276,-1.749437,10.548459,1.738718e-01,3.811864e+04,1.402335,0.160815,2.636523,KATZ,DIST7_5_NEAR_RATIO
0,CLOSENESS_CENTRALITY*DIST7_5_NEAR_RATIO,3.505747,3.330630e+01,2.228708,-0.862440,7.873934,4.221306e-01,2.627883e+03,1.572995,0.115720,3.111291,CLOSENESS_CENTRALITY,DIST7_5_NEAR_RATIO
0,BETWEENNESS_CENTRALITY*DIST7_5_NEAR_RATIO,37.924749,2.954670e+16,35.573175,-31.797394,107.646891,1.550841e-14,5.629254e+46,1.066105,0.286376,1.804017,BETWEENNESS_CENTRALITY,DIST7_5_NEAR_RATIO
0,CLUSTCOEF*DIST7_5_NEAR_RATIO,0.482621,1.620316e+00,0.257615,-0.022295,0.987537,9.779520e-01,2.684613e+00,1.873421,0.061010,4.034805,CLUSTCOEF,DIST7_5_NEAR_RATIO
0,AGE7*DIST7_5_NEAR_RATIO,0.016759,1.016900e+00,0.004806,0.007340,0.026178,1.007367e+00,1.026524e+00,3.487171,0.000488,11.000362,AGE7,DIST7_5_NEAR_RATIO


In [88]:
results_df[results_df['term 1']=='AGE7']

,covariate,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p),term 1,term 2
0,AGE7*SEX_SHANNON,0.005062,1.005075,0.002681,-0.000193,0.010318,0.999807,1.010371,1.888068,5.901687e-02,4.082729,AGE7,SEX_SHANNON
0,AGE7*RELTYPE_SHANNON,-0.001214,0.998787,0.002426,-0.005968,0.003540,0.994050,1.003547,-0.500413,6.167846e-01,0.697161,AGE7,RELTYPE_SHANNON
0,AGE7*ALTERTYPE_SHANNON,-0.002891,0.997113,0.002565,-0.007918,0.002136,0.992113,1.002138,-1.127169,2.596712e-01,1.945242,AGE7,ALTERTYPE_SHANNON
0,AGE7*AGE65_SHANNON,-0.000374,0.999626,0.002454,-0.005184,0.004436,0.994830,1.004446,-0.152405,8.788674e-01,0.186283,AGE7,AGE65_SHANNON
0,AGE7*AGE_Q_SHANNON,0.006801,1.006824,0.002456,0.001987,0.011615,1.001989,1.011682,2.768874,5.625030e-03,7.473924,AGE7,AGE_Q_SHANNON
0,AGE7*AGE_10_SHANNON,0.002574,1.002577,0.002463,-0.002253,0.007401,0.997750,1.007428,1.045123,2.959663e-01,1.756495,AGE7,AGE_10_SHANNON
0,AGE7*SNI_TOT_SHANNON,0.001911,1.001913,0.002605,-0.003194,0.007016,0.996812,1.007041,0.733835,4.630493e-01,1.110762,AGE7,SNI_TOT_SHANNON
0,AGE7*EDU_COLLEGE_SHANNON,0.003693,1.003700,0.002422,-0.001053,0.008439,0.998948,1.008475,1.525118,1.272298e-01,2.974492,AGE7,EDU_COLLEGE_SHANNON
0,AGE7*EDU_HS_SHANNON,0.003420,1.003426,0.002598,-0.001672,0.008512,0.998329,1.008549,1.316391,1.880427e-01,2.410868,AGE7,EDU_HS_SHANNON
0,AGE7*SEX,0.007032,1.007057,0.002448,0.002235,0.011830,1.002237,1.011900,2.872925,4.066905e-03,7.941853,AGE7,SEX


In [89]:
sig_df = results_df[results_df['p']<=.05][['term 1', 'term 2','exp(coef)','p']]

In [91]:
sig_df[sig_df['exp(coef)']<.1]

,term 1,term 2,exp(coef),p


In [92]:
sig_df[sig_df['exp(coef)']>1.5]

,term 1,term 2,exp(coef),p
0,DEGREE_CENTRALITY,AGE7,1.448470e+01,3.937340e-02
0,DEGREE_CENTRALITY,APOE_risk,1.452446e+204,1.507791e-05
0,DEGREE_CENTRALITY,CVD_HISTORY,2.711558e+146,2.565702e-03
0,DEGREE_CENTRALITY,CSD-D_loneliness_item,9.468762e+78,3.344507e-02
0,DEGREE_CENTRALITY,APOE4_+_RATIO,1.235938e+254,3.045070e-02
0,CONSTRAINT,APOE_risk,1.679530e+00,3.598996e-02
0,EIGENVECTOR,APOE_risk,8.365406e+18,1.117468e-06
0,KATZ,CSD-D_loneliness_item,9.142110e+10,5.716146e-05
0,CLOSENESS_CENTRALITY,AGE_Q_SHANNON,1.323378e+02,5.321908e-03
0,CLOSENESS_CENTRALITY,APOE_risk,9.113016e+03,9.749279e-06
